In [1]:
# import 
import json
import sys

import pandas

In [295]:
# setting
## file name
FILENAME = './summary.json'
COLUMNS = [
    'title', 'date', 'initial', 'kind', # '2019u',
    '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
    'average_all', 'average', 'average_exclude_debut', 'highest', 'highest_exclude_debut',
    'over30', 'over50', 'over100', 'over200', 'under30', 'under50', 'under100', 'under200', 'count_all', 'count'
]
URA = '2019'
RANK_ALL = ['2005', '2006#1', '2006#2', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
RANK = RANK_ALL[3:]

def year_filter(value: str, initial: str, array: list, mode: str = 'ge'):
    if mode not in ['gt', 'ge']:
        raise ValueError(f'invalid mode: {mode}')
    if value in array and initial in array:
        if mode == 'ge':
            return array.index(value) >= array.index(initial)
        else:
            return array.index(value) > array.index(initial)
    elif value in array:
        return True
    else:
        return False

In [233]:
# read json
with open(FILENAME, encoding='utf8') as summary_file:
    summary = json.load(summary_file)

In [285]:
# make main dataframe
df = pandas.DataFrame(summary)
df.index = df.no
df['average_all'] = df.average
df = df.drop(columns=['no', 'average'], axis=0)
df.date = pandas.to_datetime(df.date)
df.head()

,title,ruby,inst,hall_of_fame,works,debut,date,2019u,initial,kind,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,average_all
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV (桜の花は乱れ咲き), CLASS...",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,2005,"[SA, OA, OT]",...,96.0,123.0,68.0,169.0,165.0,100.0,134.0,164.0,123.0,121.462
2,青蛾月,None,False,False,"[幻想庭園, étoiles (SEIGAGETSU)]",幻想庭園,1988-01-25,22.0,2005,"[SA, OA]",...,72.0,57.0,54.0,87.0,81.0,59.0,146.0,126.0,54.0,80.923
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,2005,"[SA, OA, S-2]",...,175.0,141.0,218.0,141.0,238.0,190.0,155.0,164.0,216.0,171.769
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,2005,[OA],...,204.0,131.0,174.0,194.0,253.0,244.0,202.0,250.0,260.0,201.077
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,2005,"[SA, OA]",...,81.0,96.0,150.0,106.0,156.0,244.0,187.0,63.0,106.0,125.615


* average  
平均順位  
注) データが揃っている2007年以降
* average_exclude_debut  
デビュー年以外の平均順位
* highest  
最高順位
* highest_exclude_debut  
デビュー年以外の最高順位
* over30  
30位を超えた回数
* over50  
50位を超えた回数
* over100  
100位を超えた回数
* over200  
200いを超えた回数
* count_all  
登場回数
* count  
データが揃っている登場回数

In [296]:
df['average'] = 0.
df['average_exclude_debut'] = 0.
df['highest'] = 0.
df['highest_exclude_debut'] = 0.
df['over30'] = 0
df['over50'] = 0
df['over100'] = 0
df['over200'] = 0
df['under30'] = 0
df['under50'] = 0
df['under100'] = 0
df['under200'] = 0
df['count_all'] = 0
df['count'] = 0

for no in df.index:
    song = df.loc[no]
    to_mean = list(filter(lambda y: year_filter(y, song.initial, RANK), RANK))
    to_mean_exclude_debut = list(filter(lambda y: year_filter(y, song.initial, RANK, 'gt'), RANK))
    df.at[no, 'average'] = song[to_mean].mean()
    df.at[no, 'average_exclude_debut'] = song[to_mean_exclude_debut].mean()
    df.at[no, 'highest_exclude_debut'] = song[to_mean_exclude_debut].min()
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL), RANK_ALL))
    df.at[no, 'highest'] = song[to_mean_all].min()
    df.at[no, 'over30'] = len(list(filter(lambda s: s > 30, song[to_mean_all].values)))
    df.at[no, 'over50'] = len(list(filter(lambda s: s > 50, song[to_mean_all].values)))
    df.at[no, 'over100'] = len(list(filter(lambda s: s > 100, song[to_mean_all].values)))
    df.at[no, 'over200'] = len(list(filter(lambda s: s > 200, song[to_mean_all].values)))
    df.at[no, 'under30'] = len(list(filter(lambda s: s <= 30, song[to_mean_all].values)))
    df.at[no, 'under50'] = len(list(filter(lambda s: s <= 50, song[to_mean_all].values)))
    df.at[no, 'under100'] = len(list(filter(lambda s: s <= 100, song[to_mean_all].values)))
    df.at[no, 'under200'] = len(list(filter(lambda s: s <= 200, song[to_mean_all].values)))
    df.at[no, 'count_all'] = len(to_mean_all)
    df.at[no, 'count'] = len(to_mean)

df.head()
# df.iloc[145]

,title,ruby,inst,hall_of_fame,works,debut,date,2019u,initial,kind,...,over30,over50,over100,over200,under30,under50,under100,under200,count_all,count
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV (桜の花は乱れ咲き), CLASS...",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,2005,"[SA, OA, OT]",...,13,13,9,0,0,0,4,13,16,13
2,青蛾月,None,False,False,"[幻想庭園, étoiles (SEIGAGETSU)]",幻想庭園,1988-01-25,22.0,2005,"[SA, OA]",...,13,13,2,0,0,0,11,13,16,13
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,2005,"[SA, OA, S-2]",...,13,13,13,3,0,0,0,10,16,13
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,2005,[OA],...,13,13,13,6,0,0,0,7,16,13
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,2005,"[SA, OA]",...,13,13,9,1,0,0,4,12,16,13


## Hall of Fame

In [267]:
df.query('hall_of_fame == True')

,title,ruby,inst,hall_of_fame,works,debut,date,2019u,initial,kind,...,over50,over100,over200,under30,under50,under100,under200,count_all,count,highest_all
no,,,,,,,,,,,,,,,,,,,,,
125,愛と誠,None,False,True,"[Dilettante, Grand Finale, 愛と誠 ～YAMATO＆LOVE×××]",Dilettante,2005-06-22,NaN,2005,"[SA, OA, CA]",...,0,0,0,10,10,10,10,16,13,1.0
143,亡國覚醒カタルシス,None,False,True,"[亡國覚醒カタルシス, .hack//Roots O.S.T. (TV size), COL...",亡國覚醒カタルシス,2006-05-24,NaN,2006#2,"[S-1, OT, ST, CA]",...,0,0,0,5,5,5,5,14,13,1.0
279,リュウコウセカイ,None,False,True,[流行世界],流行世界,2014-08-27,NaN,2014,[OA],...,0,0,0,3,3,3,3,6,13,1.0


In [277]:
df.query('highest == 1 and hall_of_fame == False')

,title,ruby,inst,hall_of_fame,works,debut,date,2019u,initial,kind,...,over50,over100,over200,under30,under50,under100,under200,count_all,count,highest_all
no,,,,,,,,,,,,,,,,,,,,,
99,戦争と平和,None,False,False,"[EROTIC & HERETIC, Grand Finale, 愛と誠 ～YAMATO＆L...",EROTIC & HERETIC,2002-07-24,NaN,2005,"[SA, OA, CA]",...,0,0,0,14,16,16,16,16,13,1.0
132,北京LOVERS,None,False,False,"[Dilettante, 血と蜜 ～Anthology of Gothic Lolita＆H...",Dilettante,2005-06-22,NaN,2005,"[OA, CA]",...,0,0,0,15,15,15,15,16,13,1.0
135,聖少女領域,None,False,False,"[聖少女領域, ローゼンメイデン・トロイメント オリジナルサウンドトラック (TV size...",聖少女領域,2005-10-26,NaN,2006#1,"[S-1, OT, ST, CA]",...,0,0,0,15,15,15,15,15,13,1.0
152,勇侠青春謳,None,False,False,"[勇侠青春謳, コードギアス 反逆のルルーシュ O.S.T. (TV size), 桂冠詩人...",勇侠青春謳,2006-10-25,NaN,2006#2,"[S-1, OT, OM, ST, CA]",...,0,0,0,13,14,14,14,14,13,1.0
326,芸術変態論,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,2018,[OA],...,0,0,0,2,2,2,2,2,13,1.0
338,BArADiPArADicA,None,False,False,[Fantasia],Fantasia,2019-08-28,NaN,2019,[OA],...,0,0,0,1,1,1,1,1,13,1.0


# MAIN ranking

In [297]:
result = df.query('hall_of_fame == False').loc[:]
result['rank_average'] = result.average.rank(method='min', na_option='bottom')
result['rank_average_exclude_debut'] = result.average_exclude_debut.rank(method='min', na_option='bottom')
result['rank_highest'] = result.highest.rank(method='min', na_option='bottom')
result['rank_highest_exclude_debut'] = result.highest_exclude_debut.rank(method='min', na_option='bottom')
result.head()

,title,ruby,inst,hall_of_fame,works,debut,date,2019u,initial,kind,...,under30,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV (桜の花は乱れ咲き), CLASS...",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,2005,"[SA, OA, OT]",...,0,0,4,13,16,13,157.0,135.0,194.0,149.0
2,青蛾月,None,False,False,"[幻想庭園, étoiles (SEIGAGETSU)]",幻想庭園,1988-01-25,22.0,2005,"[SA, OA]",...,0,0,11,13,16,13,97.0,75.0,168.0,120.0
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,2005,"[SA, OA, S-2]",...,0,0,0,10,16,13,229.0,213.0,288.0,258.0
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,2005,[OA],...,0,0,0,7,16,13,286.0,262.0,274.0,245.0
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,2005,"[SA, OA]",...,0,0,4,12,16,13,165.0,142.0,183.0,137.0


In [269]:
average_top = result.sort_values(['rank_average_exclude_debut', 'date'], ascending=[True, True]) \
                    .query('average_exclude_debut <= 100') \
                    [['title', 'rank_average_exclude_debut', 'average_exclude_debut', 'date']].astype({'rank_average_exclude_debut': int})

average_top

,title,rank_average_exclude_debut,average_exclude_debut,date
no,,,,
338,BArADiPArADicA,1,1.000000,2019-08-28
343,自由戀愛,2,2.000000,2019-08-28
326,芸術変態論,3,3.500000,2018-07-25
135,聖少女領域,4,3.923077,2005-10-26
107,月蝕グランギニョル,5,4.846154,2003-11-05
...,...,...,...,...
301,快楽のススメ,122,95.800000,2015-09-09
70,月光夜,123,96.461538,1998-11-21
328,ヤマトイズム,124,98.000000,2018-07-25


In [274]:
highest_top = result.sort_values(['rank_highest', 'date'], ascending=[True, True]) \
                    .query('highest <= 50') \
                    [['title', 'rank_highest', 'highest', 'date']].astype({'rank_highest': int, 'highest': int})

highest_top

,title,rank_highest,highest,date
no,,,,
99,戦争と平和,1,1,2002-07-24
132,北京LOVERS,1,1,2005-06-22
135,聖少女領域,1,1,2005-10-26
152,勇侠青春謳,1,1,2006-10-25
326,芸術変態論,1,1,2018-07-25
...,...,...,...,...
82,閉ざされた画室,155,48,2001-04-25
267,乙女の贖い,155,48,2013-07-24
15,メガロポリス・アリス -MEGALOPOLIS ALICE-,158,49,1992-12-09


In [255]:
highest_top = result.sort_values(['rank_highest_exclude_debut', 'date'], ascending=[True, True]) \
                    .query('highest_exclude_debut <= 50') \
                    [['title', 'rank_highest_exclude_debut', 'highest_exclude_debut', 'date']].astype({'rank_highest_exclude_debut': int, 'highest_exclude_debut': int})

highest_top

,title,rank_highest_exclude_debut,highest_exclude_debut,date
no,,,,
132,北京LOVERS,1,1,2005-06-22
135,聖少女領域,1,1,2005-10-26
107,月蝕グランギニョル,3,2,2003-11-05
179,逢魔ヶ恋,3,2,2007-10-03
145,GOD DIVA,5,3,2006-06-28
...,...,...,...,...
271,薔薇娼館,103,48,2013-09-11
15,メガロポリス・アリス -MEGALOPOLIS ALICE-,109,49,1992-12-09
48,ピアニィ・ピンク,110,50,1997-05-21


In [304]:
tmp = result.loc[:]
tmp['under10'] = 0
for no in tmp.index:
    song = tmp.loc[no]
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL), RANK_ALL))
    tmp.at[no, 'under10'] = len(list(filter(lambda s: s <= 10, song[to_mean_all].values)))
tmp['under10_mean'] = tmp.under10 / tmp.count_all
tmp.sort_values(['under10_mean', 'date'], ascending=[False, True]) \
   .query('under10_mean >= 1') \
   [['title', 'under10', 'under10_mean', 'date']]

,title,under10,under10_mean,date
no,,,,
107,月蝕グランギニョル,16,1.0,2003-11-05
135,聖少女領域,15,1.0,2005-10-26
191,雪華懺悔心中,11,1.0,2008-08-27
326,芸術変態論,2,1.0,2018-07-25
338,BArADiPArADicA,1,1.0,2019-08-28
343,自由戀愛,1,1.0,2019-08-28


In [303]:
tmp = result.loc[:]
tmp['under20'] = 0
for no in tmp.index:
    song = tmp.loc[no]
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL), RANK_ALL))
    tmp.at[no, 'under20'] = len(list(filter(lambda s: s <= 20, song[to_mean_all].values)))
tmp['under20_mean'] = tmp.under20 / tmp.count_all
tmp.sort_values(['under20_mean', 'date'], ascending=[False, True]) \
   .query('under20_mean >= 1') \
   [['title', 'under20', 'under20_mean', 'date']]

,title,under20,under20_mean,date
no,,,,
107,月蝕グランギニョル,16,1.0,2003-11-05
108,未來のイヴ,16,1.0,2003-11-05
135,聖少女領域,15,1.0,2005-10-26
191,雪華懺悔心中,11,1.0,2008-08-27
282,Lolicate,6,1.0,2014-08-27
322,卑弥呼外伝,3,1.0,2017-03-29
326,芸術変態論,2,1.0,2018-07-25
338,BArADiPArADicA,1,1.0,2019-08-28
339,21世紀新青年,1,1.0,2019-08-28


In [256]:
tmp = result.loc[:]
tmp['under30_mean'] = tmp.under30 / tmp.count_all
tmp.sort_values(['under30_mean', 'date'], ascending=[False, True]) \
   .query('under30_mean >= 1') \
   [['title', 'under30', 'under30_mean', 'date']]

,title,under30,under30_mean,date
no,,,,
107,月蝕グランギニョル,16,1.0,2003-11-05
108,未來のイヴ,16,1.0,2003-11-05
135,聖少女領域,15,1.0,2005-10-26
191,雪華懺悔心中,11,1.0,2008-08-27
251,凶夢伝染,8,1.0,2012-01-25
282,Lolicate,6,1.0,2014-08-27
319,まだら恋椿外道,4,1.0,2016-08-24
322,卑弥呼外伝,3,1.0,2017-03-29
326,芸術変態論,2,1.0,2018-07-25


In [305]:
tmp = result.loc[:, :]
tmp['under50_mean'] = tmp.under50 / tmp.count_all
tmp.sort_values(['under50_mean', 'date'], ascending=[False, True]) \
   .query('under50_mean >= 1') \
   [['title', 'under50', 'under50_mean', 'date']]

,title,under50,under50_mean,date
no,,,,
99,戦争と平和,16,1.0,2002-07-24
107,月蝕グランギニョル,16,1.0,2003-11-05
108,未來のイヴ,16,1.0,2003-11-05
112,地獄の季節,16,1.0,2003-12-03
121,禁じられた遊び,16,1.0,2004-10-22
135,聖少女領域,15,1.0,2005-10-26
145,GOD DIVA,14,1.0,2006-06-28
149,KING KNIGHT,14,1.0,2006-09-21
152,勇侠青春謳,14,1.0,2006-10-25


In [257]:
tmp = result.loc[:, :]
tmp['under100_mean'] = tmp.under100 / tmp.count_all
tmp.sort_values(['under100_mean', 'date'], ascending=[False, True]) \
   .query('under100_mean >= 1') \
   [['title', 'under100', 'under100_mean', 'date']]

,title,under100,under100_mean,date
no,,,,
24,嵐ヶ丘,16,1.0,1993-06-09
72,コッペリアの柩,16,1.0,1999-12-24
99,戦争と平和,16,1.0,2002-07-24
107,月蝕グランギニョル,16,1.0,2003-11-05
108,未來のイヴ,16,1.0,2003-11-05
112,地獄の季節,16,1.0,2003-12-03
121,禁じられた遊び,16,1.0,2004-10-22
126,人生美味礼讃,16,1.0,2005-06-22
135,聖少女領域,15,1.0,2005-10-26


# URA ranking

In [258]:
RANK = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

# ura ranking
ura = df[df[URA] > 100][COLUMNS]
ura['rank_average'] = ura.average.rank(method='min', na_option='bottom')
ura['rank_average_exclude_debut'] = ura.average_exclude_debut.rank(method='min', na_option='bottom')
ura['rank_highest'] = ura.highest.rank(method='min', na_option='bottom')
ura['rank_highest_exclude_debut'] = ura.highest_exclude_debut.rank(method='min', na_option='bottom')

ura.head()

,title,date,initial,kind,2007,2008,2009,2010,2011,2012,...,under30,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1997-04-25,2005,"[SA, OA, OT]",86.0,126.0,116.0,109.0,96.0,123.0,...,0,0,4,13,16,13,61.0,49.0,100.0,67.0
3,マリーゴールド・ガーデン,1988-01-25,2005,"[SA, OA, S-2]",141.0,162.0,142.0,150.0,175.0,141.0,...,0,0,0,10,16,13,128.0,123.0,185.0,167.0
4,鏡面界 im Juni,1988-01-25,2005,[OA],165.0,185.0,181.0,171.0,204.0,131.0,...,0,0,0,7,16,13,178.0,165.0,172.0,155.0
5,アンジェ・ノワールの祭戯,1988-01-25,2005,"[SA, OA]",109.0,131.0,111.0,93.0,81.0,96.0,...,0,0,4,12,16,13,68.0,55.0,89.0,58.0
6,紅い睡蓮の午後,1988-01-25,2005,"[SA, OA]",144.0,176.0,153.0,164.0,188.0,204.0,...,0,0,0,8,16,13,152.0,146.0,181.0,162.0


In [259]:
average_top = ura.sort_values(['rank_average_exclude_debut', 'date'], ascending=[True, True]) \
                 .query('average_exclude_debut <= 100') \
                 [['title', 'rank_average_exclude_debut', 'average_exclude_debut', 'date']].astype({'rank_average_exclude_debut': int})

average_top

,title,rank_average_exclude_debut,average_exclude_debut,date
no,,,,
185,神風,1,39.400000,2008-08-27
261,真夏の憂愁夫人,2,44.857143,2012-07-18
269,ローズ家の双子達,3,52.333333,2013-09-11
268,令嬢薔薇図鑑,4,59.166667,2013-09-11
130,昭和恋々幻燈館,5,62.692308,2005-06-22
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,6,63.222222,2009-08-26
195,鬼帝の剣,7,64.300000,2008-11-19
264,快恠奇奇,8,66.500000,2013-02-20
181,コトダマ,9,67.000000,2008-01-23


In [260]:
for no in average_top.index:
    print(f'{average_top.at[no, "rank_average_exclude_debut"]}|{average_top.at[no, "title"]}|{average_top.at[no, "average_exclude_debut"]:.2f}')

1|神風|39.40
2|真夏の憂愁夫人|44.86
3|ローズ家の双子達|52.33
4|令嬢薔薇図鑑|59.17
5|昭和恋々幻燈館|62.69
6|処女懐胎、あるいは白骨塔より少女達は飛翔する|63.22
7|鬼帝の剣|64.30
8|快恠奇奇|66.50
9|コトダマ|67.00
10|黒百合隠密カゲキダン|69.83
11|堕天國宣戦|72.67
12|昭和B級下手喰い道|78.33
13|暗黒天国|78.64
14|お毒味LADY|79.89
15|女化生舞楽図|83.67
16|亂世エロイカ|86.12
17|六道輪廻サバイバル|86.18
18|ヘテロ失楽園|88.50
19|陸と海と空と|89.25
20|上海繚乱ロマンチカ|90.78
21|恋せよ乙女 -Love story of ZIPANG-|91.85
22|地獄の門|93.33
23|月光夜|96.46
24|平成日本残酷物語|96.62


In [261]:
highest_top = ura.sort_values(['rank_highest_exclude_debut', 'date'], ascending=[True, True]) \
                 .query('highest_exclude_debut <= 50') \
                 [['title', 'rank_highest_exclude_debut', 'highest_exclude_debut', 'date']].astype({'rank_highest_exclude_debut': int, 'highest_exclude_debut': int})

highest_top

,title,rank_highest_exclude_debut,highest_exclude_debut,date
no,,,,
261,真夏の憂愁夫人,1,9,2012-07-18
274,黒百合隠密カゲキダン,2,10,2013-09-11
269,ローズ家の双子達,3,12,2013-09-11
185,神風,4,14,2008-08-27
215,堕天國宣戦,4,14,2009-10-21
206,お毒味LADY,6,18,2009-08-26
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,7,19,2009-08-26
164,暗黒天国,8,20,2007-05-23
195,鬼帝の剣,9,23,2008-11-19


In [262]:
tmp = ura.loc[:, :]
tmp['under100_mean'] = tmp.under100 / tmp.count_all
tmp.sort_values(['under100_mean', 'date'], ascending=[False, True]) \
   .query('under100_mean >= 0.8') \
   [['title', 'under100', 'under100_mean', 'date']]

,title,under100,under100_mean,date
no,,,,
181,コトダマ,11,0.916667,2008-01-23
185,神風,10,0.909091,2008-08-27
261,真夏の憂愁夫人,7,0.875000,2012-07-18
264,快恠奇奇,6,0.857143,2013-02-20
268,令嬢薔薇図鑑,6,0.857143,2013-09-11
269,ローズ家の双子達,6,0.857143,2013-09-11
164,暗黒天国,10,0.833333,2007-05-23
175,六道輪廻サバイバル,10,0.833333,2007-08-22
195,鬼帝の剣,9,0.818182,2008-11-19


In [263]:
# 未投票
zero = ['日曜日のシエスタ', 'La vie en rose', 'Féliciter', 'Nocturnal', '寶島', '逢魔蛾城の伯爵', '小夜すみれ ～ある夜の森の小さなお話', '魂ノ代']
ura.query('title in @zero')

,title,date,initial,kind,2007,2008,2009,2010,2011,2012,...,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
55,日曜日のシエスタ,1997-12-17,2005,"[SA, ST]",129.0,208.0,179.0,171.0,169.0,139.0,...,0,0,7,16,13,158.0,149.0,168.0,151.0,0.0
91,La vie en rose,2001-07-25,2005,[OA],187.0,196.0,221.0,155.0,211.0,217.0,...,0,0,4,16,13,199.0,195.0,193.0,181.0,0.0
106,Féliciter,2003-04-23,2005,[SA],168.0,174.0,215.0,179.0,223.0,185.0,...,0,0,5,16,13,182.0,173.0,203.0,191.0,0.0
180,Nocturnal,2007-12-12,2008,[SA],NaN,154.0,202.0,192.0,223.0,217.0,...,0,0,2,12,12,207.0,207.0,190.0,207.0,0.0
216,寶島,2009-10-21,2010,"[S-2, CA]",NaN,NaN,NaN,109.0,147.0,171.0,...,0,0,4,10,10,190.0,201.0,147.0,173.0,0.0
247,逢魔蛾城の伯爵,2011-06-29,2011,[SA],NaN,NaN,NaN,NaN,123.0,158.0,...,0,0,4,9,9,163.0,164.0,165.0,168.0,0.0
296,小夜すみれ ～ある夜の森の小さなお話,2015-01-21,2015,[SA],NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,2,5,5,194.0,194.0,189.0,177.0,0.0
311,魂ノ代,2015-10-21,2015,[S-2],NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,5,5,213.0,213.0,206.0,213.0,0.0


In [264]:
ura[ura.kind.apply(lambda k: 'S-1' in k or 'S-2' in k)]

,title,date,initial,kind,2007,2008,2009,2010,2011,2012,...,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
3,マリーゴールド・ガーデン,1988-01-25,2005,"[SA, OA, S-2]",141.0,162.0,142.0,150.0,175.0,141.0,...,0,0,10,16,13,128.0,123.0,185.0,167.0,0.000000
12,フラワーチャイルド,1988-07-25,2005,"[OA, S-1, CA]",187.0,197.0,221.0,214.0,223.0,217.0,...,0,0,5,16,13,185.0,178.0,175.0,156.0,0.000000
13,恋せよ乙女 -Love story of ZIPANG-,1992-07-07,2005,"[OA, S-1, SA, CA]",36.0,48.0,58.0,74.0,87.0,71.0,...,3,10,13,16,13,29.0,21.0,50.0,22.0,0.625000
25,舞踏会の手帖,1993-06-09,2005,"[OA, S-2, CA]",92.0,137.0,174.0,171.0,223.0,153.0,...,0,1,10,16,13,143.0,135.0,128.0,104.0,0.062500
27,エスカルゴ嬉遊曲,1994-01-19,2005,"[S-2, CA]",56.0,66.0,91.0,109.0,155.0,194.0,...,0,3,9,16,13,118.0,107.0,79.0,46.0,0.187500
37,Nous Deux C'est Pour La Vie,1995-10-18,2005,"[S-2, CA]",44.0,65.0,80.0,118.0,117.0,107.0,...,2,5,14,16,13,63.0,51.0,59.0,29.0,0.312500
42,Only Love Song,1996-08-01,2005,[S-2],187.0,129.0,185.0,150.0,158.0,217.0,...,0,0,10,16,13,141.0,133.0,168.0,151.0,0.000000
43,Wish,1996-12-24,2005,"[S-1, OT, ST, CA]",53.0,71.0,81.0,72.0,86.0,127.0,...,1,6,13,16,13,55.0,45.0,69.0,36.0,0.375000
44,夢のあとに ～après un rêve～,1996-12-24,2005,"[S-2, ST, OT, CA]",168.0,199.0,198.0,164.0,211.0,217.0,...,0,0,6,16,13,186.0,181.0,202.0,189.0,0.000000


In [265]:
ura_list[(160 <= ura_list[2018]) & (ura_list[2018] <= 210)]

NameError: name 'ura_list' is not defined

In [134]:
len(_)

53

In [124]:
my = ['桜の花は狂い咲き', '乙女の祈り', 'Rose Moon', '少女貴族', '熱帯性植物園', '遊月恋歌', '少女殉血', 'Animals on the Earth', '四神獣飼殺し', 'ALICE同罪イノセント', '陸と海と空と', '緋ノ糸輪廻ノGEMINI']
result.query('title in @my').join(ura_list[2018])

,title,release,kind,average,average_exclude_debut,rank_average,rank_average_exclude_debut,highest,highest_exclude_debut,rank_highest,rank_highest_exclude_debut,under_100,over_100,over_100_average,2018
4,桜の花は狂い咲き,1988,OA/SA,117.454545,117.454545,57.0,55.0,68.0,68.0,87.0,58.0,False,4,0.333333,164.0
46,乙女の祈り,1998,OA,105.454545,105.454545,48.0,42.0,52.0,52.0,65.0,38.0,False,6,0.500000,136.0
47,Rose Moon,1998,OA,90.636364,90.636364,33.0,26.0,51.0,51.0,62.0,36.0,False,8,0.666667,133.0
53,少女貴族,2001,OA,70.363636,70.363636,17.0,11.0,24.0,24.0,22.0,6.0,False,9,0.750000,192.0
64,熱帯性植物園,2002,OA,109.090909,109.090909,50.0,44.0,51.0,51.0,62.0,36.0,False,5,0.416667,175.0
65,遊月恋歌,2002,OA/SA,41.363636,41.363636,2.0,1.0,18.0,18.0,13.0,4.0,True,11,0.916667,110.0
74,少女殉血,2003,ST,113.727273,113.727273,54.0,51.0,27.0,27.0,28.0,10.0,False,5,0.416667,126.0
121,Animals on the Earth,2009,OA,93.000000,101.285714,36.0,39.0,35.0,45.0,40.0,25.0,False,5,0.416667,175.0
141,四神獣飼殺し,2010,OA,115.142857,129.833333,56.0,63.0,27.0,71.0,28.0,68.0,False,4,0.333333,115.0
159,ALICE同罪イノセント,2012,OA,72.333333,85.200000,20.0,20.0,8.0,35.0,6.0,20.0,False,5,0.416667,119.0


In [122]:
# for idx, i in enumerate(map(lambda line: f'<p>{line}</p>', ura_list.title)):
#     print(i, end='')
#     if idx % 50 == 49:
#         print()

In [123]:
# for idx, i in enumerate(map(lambda line: f'<li>{line}</li>', ura_list.title)):
#     print(i, end='')
#     if idx % 100 == 99:
#         print()